# 第四章 深度学习：背景和工具

本章依赖的类库有：

- numpy 快速操作结构数组的工具
- keras 二次封装的深度学习库
- tensorflow 深度学习库
- caffe 深度学习库

python类库安装教程见实体书附录A，深度学习库安装教程见实体书附录B，每个小节可以独立运行。

____

## 4.3 深度学习框架快速上手

### Keras完成逻辑分类


In [1]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils

batch_size = 128 # 梯度下降批数据量
nb_classes = 10 # 类别
nb_epoch = 10 # 梯度下降epoch循环次数，每次循环包含全部训练样本
img_size = 28 * 28 # 输入图片大小

# 加载数据，已执行shuffle-split（训练-测试集随机分割）
(X_train, y_train), (X_test, y_test) = mnist.load_data()

Using TensorFlow backend.


In [2]:
# 以Tensorflow为后端，归一化输入数据，生成图片向量
X_train = X_train.reshape(y_train.shape[0], img_size).astype('float32') / 255
X_test = X_test.reshape(y_test.shape[0], img_size).astype('float32') / 255

X_train.shape, X_test.shape

((60000, 784), (10000, 784))

In [3]:
# One-Hot编码标签，将如[3,2,...] 编码成[[0,0,0,1,0,0,0,0,0,0], [0,0,1,0,0,0,0,0,0,0],...]
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [4]:
# 创建模型，逻辑分类相当于一层全连接的神经网络(Dense是Keras中定义的DNN模型)
model = Sequential([
Dense(10, input_shape=(img_size,), activation='softmax'),
])

In [5]:
# 编译模型；rmsprop是改进的梯度下降，你暂时不需要了解
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [6]:
# 训练
model.fit(X_train, Y_train,
          batch_size=batch_size, epochs=nb_epoch,
          verbose=1, validation_data=(X_test, Y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 1s - loss: 0.6068 - acc: 0.8485 - val_loss: 0.3410 - val_acc: 0.9077

In [7]:
# 测试
score = model.evaluate(X_test, Y_test, verbose=0)
print('accuracy: {}'.format(score[1]))

accuracy: 0.9255


### Keras的引擎：Tensorflow

In [8]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./data/MNIST', one_hot=True)

Extracting ./data/MNIST/train-images-idx3-ubyte.gz
Extracting ./data/MNIST/train-labels-idx1-ubyte.gz
Extracting ./data/MNIST/t10k-images-idx3-ubyte.gz
Extracting ./data/MNIST/t10k-labels-idx1-ubyte.gz


In [9]:
x = tf.placeholder("float", [None, 784]) # 输入占位符
W = tf.Variable(tf.zeros([784,10])) # 模型变量
b = tf.Variable(tf.zeros([10])) # 模型变量
y = tf.nn.softmax(tf.matmul(x,W) + b) # 模型

y_pred = tf.placeholder("float", [None,10]) # 输出占位符
cross_entropy = - tf.reduce_sum(y_pred * tf.log(y)) # 交叉熵损失函数
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy) # 优化器，以0.01的学习速率最小化交叉熵

In [10]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [11]:
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(128)
    sess.run(train_step, feed_dict={x: batch_xs, y_pred: batch_ys})

In [12]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_pred, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(accuracy, feed_dict={x: mnist.test.images, y_pred: mnist.test.labels})

0.9174

## 4.4 Caffe实现逻辑分类模型

### Caffe训练MNIST概览

见第四章目录下文件。